1. Environment Setup

In [ ]:
!pip install mtcnn tensorflow==2.12

import os
import numpy as np
import tensorflow as tf
from mtcnn import MTCNN
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, callbacks

2. Data Acquisition

2.1 FER2013 (48×48 greyscale, 7 emotions)
Where to download: Kaggle’s FER-2013 dataset → download the CSV or image files from Kaggle
Kaggle

Link: https://www.kaggle.com/datasets/msambare/fer2013

2.2 CAFE (Child Affective Facial Expression Set)
Where to download: Databrary (requires researcher registration)
childstudycenter

Link: https://databrary.org/volume/30

2.3 RaFD (Radboud Faces Database)**
Where to download: Official Radboud site (free for non-commercial academic use)
Radboud Faces Database

Link: http://www.socsci.ru.nl:8180/RaFD2/RaFD

3. Drive Mount & Storage
Mount Google Drive:

```
from google.colab import drive
drive.mount('/content/drive')
```

>This makes your Drive files available under /content/drive/MyDrive/


Folder Structure:
```
/content/drive/MyDrive/datasets/
    fer2013/
    cafe/
    rafd/
```


In [ ]:
import zipfile
from pathlib import Path

# 1. Paths to your zip files
zip_paths = [
    "/content/FER-2013_v1.zip",
    "/content/FER-2013_v2.zip",
    "/content/archive.zip",
    "/content/fer2013.csv.zip"
]

# 2. Create the dataset directory
dataset_dir = Path("/content/dataset")
dataset_dir.mkdir(parents=True, exist_ok=True)

# 3. Extract each zip
for zp in zip_paths:
    with zipfile.ZipFile(zp, 'r') as archive:
        archive.extractall(dataset_dir)
        print(f"Extracted {zp} ➔ {dataset_dir}")


Extracted /content/FER-2013_v1.zip ➔ /content/dataset
Extracted /content/FER-2013_v2.zip ➔ /content/dataset
Extracted /content/archive.zip ➔ /content/dataset
Extracted /content/fer2013.csv.zip ➔ /content/dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. Data Generators**
>Use Keras’s flow_from_directory() to read labeled subfolders under your train/test paths.

In [ ]:
# 2.1 Augmentation & Rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

test_datagen  = ImageDataGenerator(rescale=1./255)

# 2.2 Point to your directories
train_gen = train_datagen.flow_from_directory(
    '/content/dataset/train',        # train root folder
    target_size=(48,48),
    batch_size=32,
    class_mode='categorical'         # one-hot labels for multiple emotions
)

test_gen = test_datagen.flow_from_directory(
    '/content/dataset/test',         # test root folder
    target_size=(48,48),
    batch_size=32,
    class_mode='categorical'
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


– flow_from_directory expects subfolders per class and yields batches at training time

4. Data Preparation
>4.1 Face Detection & Cropping

In [ ]:
detector = MTCNN()
def extract_face(img):
    res = detector.detect_faces(img)
    x,y,w,h = res[0]['box']
    return img[y:y+h, x:x+w]

>>4.2 Resize & Normalize

In [ ]:
import tensorflow as tf
def preprocess(img):
    face = extract_face(img)
    face = tf.image.resize(face, [48,48]) / 255.0
    return face


5. Model Definition & Training

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(48,48,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_gen.num_classes, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


3.2 Early Stopping & Fit


In [ ]:
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=50,
    callbacks=[early_stop]
)


Epoch 1/50
898/898 [==============================] - 120s 132ms/step - loss: 1.7823 - accuracy: 0.2666 - val_loss: 1.6783 - val_accuracy: 0.3291
Epoch 2/50
898/898 [==============================] - 109s 121ms/step - loss: 1.7027 - accuracy: 0.3166 - val_loss: 1.5212 - val_accuracy: 0.4146
Epoch 3/50
898/898 [==============================] - 108s 120ms/step - loss: 1.6233 - accuracy: 0.3574 - val_loss: 1.4754 - val_accuracy: 0.4231
Epoch 4/50
898/898 [==============================] - 107s 120ms/step - loss: 1.5860 - accuracy: 0.3776 - val_loss: 1.4122 - val_accuracy: 0.4518
Epoch 5/50
898/898 [==============================] - 111s 123ms/step - loss: 1.5477 - accuracy: 0.3966 - val_loss: 1.3791 - val_accuracy: 0.4776
Epoch 6/50
898/898 [==============================] - 109s 121ms/step - loss: 1.5257 - accuracy: 0.4044 - val_loss: 1.3448 - val_accuracy: 0.4769
Epoch 7/50
898/898 [==============================] - 109s 122ms/step - loss: 1.4977 - accuracy: 0.4204 - val_loss: 1.3425 -

– Stops when validation loss stops improving to prevent overfitting

4. Evaluation & Inference
>4.1 Plot Metrics

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='test_acc')
plt.legend(); plt.show()


NameError: name 'history' is not defined

>>4.2 Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Gather all test data and true labels
X_test, y_test = [], []
for X_batch, y_batch in test_gen:
    X_test.append(X_batch)
    y_test.append(y_batch)
    if len(X_test)*test_gen.batch_size >= test_gen.samples:
        break

X_test = np.vstack(X_test)
y_true = np.argmax(np.vstack(y_test), axis=1)
y_pred = np.argmax(model.predict(X_test), axis=1)

cm = confusion_matrix(y_true, y_pred)
print(cm)
print(classification_report(y_true, y_pred, target_names=list(test_gen.class_indices.keys())))
